Use the neccessary packages

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import DataFrame
import math
import numpy
from statistics import *
from numpy import nan

Connect to Drive and read the dataset

In [ ]:
# Make suitable changes here. My .csv file is in :  My Drive/Colab Notebooks
from google.colab import drive
drive.mount("/content/drive/")
%cd 'national_park_visitation_analysis_w_mobile_phone_data/volume/data/raw/census_files/'
!pwd #Prints the present working directory

Mounted at /content/drive/
national_park_visitation_analysis_w_mobile_phone_data/volume/data/raw/census_files/
/content/drive/My Drive/RTPM496/National Park Research/Income_and_race


In [ ]:
# Make suitable changes here. My .csv file is in :  My Drive/Colab Notebooks
!pwd #Prints the present working directory
agedata = pd.read_csv('age.csv')
agedata.tail()



/content/drive/My Drive/RTPM496/National Park Research/Income_and_race


,Geo_FIPS,total.population,less.than.5,5.to.9,10.to.14,15.to.17,18.to.19,20Y,21Y,22.to.24,25.to.29,30.to.34,35.to.39,40.to.44,45.to.49,50.to.54,55.to.59,60.to.61,62.to.64,65.to.66,67.to.69,70.to.74,75.to.79,80.to.84,more.than.85,median.age,male.median.age,female.median.age
220328,721537506011,883,48,78,120,12,0,9,8,62,45,0,52,53,32,38,45,53,22,42,31,96,24,0,13,40.4,36.4,49.0
220329,721537506012,2523,227,175,96,36,79,26,93,110,141,109,154,106,180,256,101,35,61,149,76,169,98,8,38,40.3,40.1,43.5
220330,721537506013,991,86,37,162,76,40,73,0,77,53,0,38,52,145,46,89,0,0,17,0,0,0,0,0,23.9,18.6,24.9
220331,721537506021,1577,68,36,18,24,0,0,54,28,126,43,95,130,77,195,124,70,60,55,53,124,84,104,9,50.9,46.5,54.6
220332,721537506022,648,43,0,0,63,37,14,0,0,50,62,20,76,20,80,27,12,8,42,8,33,24,29,0,40.5,32.3,48.4


Extract data with missing median age information

In [ ]:
missing_age_data=agedata[agedata["median.age"].isna()]

missing_age_data=missing_age_data[["Geo_FIPS", "total.population", "less.than.5", "5.to.9",	"10.to.14",	
"15.to.17",	"18.to.19",	"20Y",	"21Y",	"22.to.24",	"25.to.29",	"30.to.34",	"35.to.39",	
"40.to.44",	"45.to.49",	"50.to.54",	"55.to.59",	"60.to.61",	"62.to.64",	"65.to.66",	"67.to.69",	
"70.to.74",	"75.to.79",	"80.to.84",	"more.than.85"]]
missing_age_data = missing_age_data[missing_age_data['total.population'] != 0]
missing_age_data.tail()


,Geo_FIPS,total.population,less.than.5,5.to.9,10.to.14,15.to.17,18.to.19,20Y,21Y,22.to.24,25.to.29,30.to.34,35.to.39,40.to.44,45.to.49,50.to.54,55.to.59,60.to.61,62.to.64,65.to.66,67.to.69,70.to.74,75.to.79,80.to.84,more.than.85
208956,530350926001,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,0,0,0,0,0,0
209887,530579901000,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,0,0,0,0,0
218511,720339800051,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0
218547,720371601001,2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
219612,721270006003,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0


We have a total of 73 missing median income in the dataset

We wish to impute the missing data using the following technique below. 

1. Find the median population of the census block group
2. Find which age bracket the median population belongs to
3. We use the center of the age bracket as the median income for the census block group. 

Step 1: Find the median population of the census block group

In [ ]:
median=(missing_age_data["total.population"]+1)/2
flooring=numpy.floor(median)
missing_age_data["median"]=flooring
missing_age_data_temp=missing_age_data.drop(columns=["Geo_FIPS", "total.population", "median"])
the_list=(missing_age_data_temp.values.tolist())
the_cum_list=[]
for i in range(0, len(the_list)):
  a_list=the_list[i]
  x=np.cumsum(a_list)
  the_cum_list.append(list(x))
missing_age_data.tail()


#type(missing_income_data["the_list"])





,Geo_FIPS,total.population,less.than.5,5.to.9,10.to.14,15.to.17,18.to.19,20Y,21Y,22.to.24,25.to.29,30.to.34,35.to.39,40.to.44,45.to.49,50.to.54,55.to.59,60.to.61,62.to.64,65.to.66,67.to.69,70.to.74,75.to.79,80.to.84,more.than.85,median
208956,530350926001,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,0,0,0,0,0,0,12.0
209887,530579901000,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,0,0,0,0,0,3.0
218511,720339800051,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,6.0
218547,720371601001,2,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
219612,721270006003,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,6.0


In [ ]:
pos=[]
check=[]
flooring_interest=list(flooring)

for i in range(0, len(the_cum_list)):
  #print(i)
  a_list=the_cum_list[i]
  interest=flooring_interest[i]
  #print(a_list)
  for n in range(0, len(a_list)):
    if (a_list[n]) < interest:
      continue
    else:
      #print(a_list)
      pos.append(n)
      check.append(a_list)
      break
len(check)




73

Step 2: Find which age bracket the median population belongs to

AND 

Step 3: We use the center of the age bracket as the median income for the census block group.

In [ ]:
pred_median=[]
for i in range(0, len(pos)):
  x=pos[i]
  if x==1:
    y=5/2
  if x==2:
    y=(5+9)/2
  if x==3:
    y=(10+ 14)/2
  if x==4:
    y=(15+ 17)/2
  if x==5:
    y=(18+ 19)/2
  if x==6:
    y=20
  if x==7:
    y=21
  if x==8:
    y=(22+24)/2
  if x==9:
    y=(25+29)/2
  if x==10:
    y=(30+ 34)/2
  if x==11:
    y=(35+ 39)/2
  if x==12:
    y=(40+44)/2
  if x==13:
    y=(45+49)/2
  if x==14: 
    y=(50+ 59)/2
  if x==15:
    y=(60+61)/2
  if x==16:
    y=(62+64)/2
  if x==17:
    y=(65+66)/2
  if x==18:
    y=(67+69)/2
  if x==19:
    y=(70+74)/2
  if x==20:
    y=(75+79)/2
  if x==21:
    y=(80+84)/2
  if x==22:
    y=85
    

  pred_median.append(y)
len(pred_median)


73

In [ ]:
missing_age_data["median.age.pred"]=pred_median
missing_age_data.head()





,Geo_FIPS,total.population,less.than.5,5.to.9,10.to.14,15.to.17,18.to.19,20Y,21Y,22.to.24,25.to.29,30.to.34,35.to.39,40.to.44,45.to.49,50.to.54,55.to.59,60.to.61,62.to.64,65.to.66,67.to.69,70.to.74,75.to.79,80.to.84,more.than.85,median,median.age.pred
2287,10970004011,3,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,23.0
2543,10970074001,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,0,0,0,0,9.0,68.0
3925,21980003001,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,0,0,0,0,7.0,68.0
5471,40131140002,28,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0,0,0,14.0,72.0
12862,60190064025,10,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,5,0,0,0,0,0,5.0,47.0


Combine with the original dataset and replace the data records with original missing median age data. 

In [ ]:
missing_age_data_total = pd.merge(agedata, missing_age_data, how="left", on=["Geo_FIPS"])
missing_age_data_total.shape



(220333, 54)

In [ ]:
final_list=[]
x=list(agedata["median.age"])
x = [str(l) for l in x]
y=list(missing_age_data_total["median.age.pred"])
y = [str(l) for l in y]

for i in range(0,len(x)):
  age=x[i]
  if age==nan or age=="nan":
    age=y[i]
    final_list.append(age)
  else:
    median=age
    final_list.append(age)



final_list = [float(l) for l in final_list]

agedata["median.age"]=final_list



,Geo_FIPS,total.population,less.than.5,5.to.9,10.to.14,15.to.17,18.to.19,20Y,21Y,22.to.24,25.to.29,30.to.34,35.to.39,40.to.44,45.to.49,50.to.54,55.to.59,60.to.61,62.to.64,65.to.66,67.to.69,70.to.74,75.to.79,80.to.84,more.than.85,median.age,male.median.age,female.median.age
220328,721537506011,883,48,78,120,12,0,9,8,62,45,0,52,53,32,38,45,53,22,42,31,96,24,0,13,40.4,36.4,49.0
220329,721537506012,2523,227,175,96,36,79,26,93,110,141,109,154,106,180,256,101,35,61,149,76,169,98,8,38,40.3,40.1,43.5
220330,721537506013,991,86,37,162,76,40,73,0,77,53,0,38,52,145,46,89,0,0,17,0,0,0,0,0,23.9,18.6,24.9
220331,721537506021,1577,68,36,18,24,0,0,54,28,126,43,95,130,77,195,124,70,60,55,53,124,84,104,9,50.9,46.5,54.6
220332,721537506022,648,43,0,0,63,37,14,0,0,50,62,20,76,20,80,27,12,8,42,8,33,24,29,0,40.5,32.3,48.4


In [ ]:
agedata.to_csv('age.csv')